<a href="https://colab.research.google.com/github/imabari/tsukuba-kdb/blob/main/tsukuba_kdb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pathlib
import json

In [2]:
import pandas as pd
import requests

In [3]:
def fetch_file(url, dir="."):

    p = pathlib.Path(dir, pathlib.PurePath(url).name)
    p.parent.mkdir(parents=True, exist_ok=True)

    r = requests.get(url)
    r.raise_for_status()

    with p.open(mode="wb") as fw:
        fw.write(r.content)
    return p

In [4]:
url = "https://github.com/Make-IT-TSUKUBA/alternative-tsukuba-kdb/raw/master/kdb_20210404.xlsx"

In [5]:
xlsx_path = fetch_file(url)

In [6]:
df = pd.read_excel(
    xlsx_path,
    skiprows=3,
    header=None,
    usecols=[0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 17, 18, 19],
)

In [7]:
df

,0,2,3,4,5,6,7,8,9,10,11,17,18,19
0,フレッシュマン・セミナー,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,科目番号,科目名,授業方法,単位数,標準履修年次,実施学期,曜時限,教室,担当教員,授業概要,備考,データ更新日,NaN,NaN
2,1101102,フレッシュマン・セミナー,2,1.0,1,春AB,金2,NaN,保呂 篤彦,大学生活について知識と理解を深め、学問研究への動機づけを高める。とくに人文学類における各主専...,人文1クラス対象\n対面,2021-02-24 19:33:14,フレッシュマン・セミナー,NaN
3,1101202,フレッシュマン・セミナー,2,1.0,1,春AB,金2,NaN,田中 友香理,大学生活について知識と理解を深め、学問研究への動機づけを高める。とくに人文学類における各主専...,人文2クラス対象\n対面,2021-02-24 19:34:03,フレッシュマン・セミナー,NaN
4,1101302,フレッシュマン・セミナー,2,1.0,1,春AB,金2,NaN,前田 修,大学生活について知識と理解を深め、学問研究への動機づけを高める。とくに人文学類における各主専...,人文3クラス対象\n対面,2021-02-24 19:34:49,フレッシュマン・セミナー,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8126,XJ21332,キャリア支援日本語 上級メディアリテラシーA,2,—,1 - 6,春C,"金3,4",NaN,山本 千波,日本の産業や経済、就職に関する情報を得るためのメディアリテラシーを身につけることを目指す。J...,外国人留学生が受講できる。 人数制限あり。(25名程度)\nオンライン(同時双方向型)\n詳...,2019-05-28 00:00:00,English Courses,NaN
8127,XJ21342,キャリア支援日本語 中級メディアリテラシーA,2,—,1 - 6,春C,"水1,2",NaN,山本 千波,日本の産業や経済、就職に関する情報を得るためのメディアリテラシーを身につけることを目指す。J...,外国人留学生が受講できる。 人数制限あり。(25名程度)\nオンライン(同時双方向型)\n詳...,2019-05-28 00:00:00,English Courses,NaN
8128,XJ21352,キャリア支援日本語 連絡コミュニケーションA,2,—,1 - 6,春C,"水5,6",NaN,金子 信子,就職活動や就業で必要な連絡(エントリーシート、eメール、電話など)の方法を身につけることを目...,外国人留学生が受講できる。 人数制限あり。(25名程度)\nオンライン(同時双方向型)\n詳...,2021-02-05 15:27:44,English Courses,NaN
8129,XJ22012,集中日本語D000A,2,—,1 - 6,春ABC,月・火・水・木・金1-4,NaN,"関崎 博紀,文 昶允,チョーハン アヌブティ",日常生活、大学生活に必要な日本語力を総合的につける。レベルによっては、プレゼンテーションやレ...,外国人留学生が受講できる。\nオンライン(同時双方向型)\n詳細は授業内で、またはmanab...,2016-01-08 00:00:00,English Courses,NaN


In [8]:
df.to_csv("data.csv")

In [9]:
df.dropna(subset=[0], inplace=True)

In [10]:
df[20] = df[0].shift()
df[20] = df[20].where(df[0] == "科目番号").fillna(method="ffill")

In [11]:
df.notnull().sum(axis=1).value_counts()

13    3821
14    2749
10     700
3      283
12     206
15     143
11      57
9       53
2       30
4       12
1        1
dtype: int64

In [12]:
df1 = df[df[0] != "科目番号"].dropna(thresh=5)

In [13]:
df1.fillna("", inplace=True)

In [14]:
df1[10] = df1[10].apply(lambda s: "\n".join([i.strip() for i in s.strip().splitlines()]))
df1[11] = df1[11].apply(lambda s: "\n".join([i.strip() for i in s.strip().splitlines()]))

In [15]:
result = []

for d in df1.values.tolist():

    if not d[13]:
        d.pop(13)

    result.append(d)

In [16]:
with open("kdb.json", "w", encoding="utf-8") as fp:
    json.dump(result, fp, indent="\t", ensure_ascii=False)